# Modelling & Analysis of Gas Turbine Blade Behavior to Predict Turbine Blade Resonance Amplitudes

# NEURAL NETWORK MODEL

IMPORTING LIBRARIES

In [14]:
import tensorflow as tf
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import os.path as path

from keras.optimizers import Adam
from keras import backend as K

MAKING A SESIION IN KERAS

In [2]:
config = tf.ConfigProto( device_count = {'GPU': 1})
#sess = tf.Session(config=config) 
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True,log_device_placement=True))
keras.backend.set_session(sess)

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device



LOADING DATASETS

In [3]:
train = pd.read_csv("traindata.csv")
train=train.sample(frac=1)
print(np.shape(train))
train.head()

(143, 8)


,N,m,P1,T1,P3,T3,TJC,A
117,98.6,75.3,13.2544,19.5,249.3,725.5,690.7,118.13
60,94.8,75.3,13.2544,27.0,249.3,725.5,637.7,78.75
96,98.6,75.3,13.1168,19.5,211.6,725.5,637.7,45.00
40,94.8,75.3,13.1168,27.0,211.6,725.5,637.7,31.50
110,98.6,75.3,13.1168,27.0,249.3,761.0,637.7,65.12


LOADING VALIDATION DATA

In [4]:
validate = pd.read_csv("validatedata.csv")
#validate = validate.sample(frac=1)
validate=validate.rename(columns={"Amplitude (Max)": "A"})
print(np.shape(validate))
validate.head()

(43, 8)


,N,m,P1,T1,P3,T3,TJC,A
0,95.84,63.78,13.16,19.59,216.63,735.61,667.44,42.9156
1,97.47,56.41,13.13,22.03,236.53,759.09,683.22,45.2331
2,96.18,64.27,13.21,26.66,225.33,759.51,673.38,65.4893
3,95.90,62.89,13.21,25.73,227.34,732.96,673.73,60.4748
4,94.88,56.32,13.15,20.78,236.98,744.53,669.48,20.7956


APPENDING BOTH TRAINING AND VALIDATION DATASET

In [5]:
train.append(validate,ignore_index=True)

,N,m,P1,T1,P3,T3,TJC,A
0,98.60,75.30,13.2544,19.50,249.30,725.50,690.70,118.1300
1,94.80,75.30,13.2544,27.00,249.30,725.50,637.70,78.7500
2,98.60,75.30,13.1168,19.50,211.60,725.50,637.70,45.0000
3,94.80,75.30,13.1168,27.00,211.60,725.50,637.70,31.5000
4,98.60,75.30,13.1168,27.00,249.30,761.00,637.70,65.1200
...,...,...,...,...,...,...,...,...
181,95.44,72.47,13.2200,20.49,237.12,760.38,686.85,82.7935
182,96.35,57.40,13.1300,26.56,224.70,753.34,646.88,42.6216
183,98.24,62.05,13.1700,20.55,224.12,759.24,642.68,58.6157
184,96.73,69.60,13.1600,24.35,224.97,747.12,688.88,83.8713


LOADING TEST DATA

In [7]:
test = pd.read_csv("testdata.csv")
test=test.rename(columns={"NL": "N","Analytical value":"A","TJ":"TJC"})
del test['Experimental value']
print(np.shape(test))
test.head()


(24, 8)


,N,m,P1,T1,P3,T3,TJC,A
0,95.0504,69.8002,13.13,25.8,222.5,752.0,685.0,76.0174
1,95.7920,71.5995,13.16,26.1,233.5,742.9,651.4,72.6498
2,96.1042,72.1003,13.12,26.1,235.2,743.5,646.3,68.9307
3,96.4946,72.4002,13.15,24.5,234.3,761.0,686.9,67.4881
4,97.3730,74.1005,13.15,25.2,243.9,747.9,672.0,68.2048


SPLITTING INTO TRAINING AND TESTING

In [9]:
features = train.columns[:7]
X_train = train[features]/100
y_train = train['A']/100
X_train=np.asarray(X_train)
y_train=np.asarray(y_train)
features = test.columns[:7]
X_test = test[features]/100
y_test = test['A']/100

In [12]:
y_train

array([1.1813, 0.7875, 0.45  , 0.315 , 0.6512, 0.4961, 0.7262, 0.4134,
       0.225 , 0.1969, 1.1813, 0.2171, 0.3308, 0.225 , 0.4961, 0.2363,
       0.1875, 0.945 , 0.7875, 1.2403, 0.1969, 0.8269, 0.1736, 0.945 ,
       0.4134, 0.1575, 0.9767, 0.4961, 0.8682, 0.1654, 0.5091, 1.0419,
       0.6946, 0.6512, 0.5625, 0.6512, 0.6615, 0.3308, 0.3   , 0.5209,
       0.4341, 0.315 , 0.1969, 1.3023, 0.4961, 0.4725, 0.3473, 0.6202,
       0.375 , 0.8682, 0.2363, 0.3938, 0.9923, 0.4725, 0.2813, 1.2713,
       0.315 , 0.3101, 0.15  , 0.315 , 0.5906, 0.6615, 0.6202, 0.4134,
       0.5906, 0.4961, 0.1688, 0.3938, 0.5625, 1.1813, 0.3308, 0.8269,
       0.63  , 0.3   , 0.6512, 0.4961, 1.2713, 0.315 , 0.4341, 0.3938,
       0.315 , 0.2953, 0.9923, 0.2481, 0.1538, 0.852 , 0.3308, 0.3938,
       0.4725, 0.75  , 1.0853, 0.6202, 0.9923, 0.3938, 0.1538, 0.5906,
       0.6202, 0.4725, 0.2067, 0.1575, 0.3101, 0.3938, 0.4341, 0.5906,
       0.3308, 0.63  , 1.1721, 0.9   , 0.2953, 1.3023, 0.375 , 0.4134,
      

NEURAL NETWORK WITH DIFFERENT VARYING PARAMETERS

In [15]:
model = Sequential()
model.add(Dense(8, input_dim=7, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model
model.fit(X_train, y_train, validation_split=0.20, epochs=1610, batch_size=10, verbose=0)
Y_pred= model.predict(X_test)
Y_pred=Y_pred*100
mse=mean_squared_error(y_test*100, Y_pred)
rmse=mse**(1/2)
print(rmse)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

7.907565521197491


In [20]:
#making rulebase
rul=pd.read_csv("Tubrulebase2layer (copy).csv",header=None)

In [21]:
rul

,0,1,2,3,4,5,6
0,0.0,0.5,0.5,0.5,0.0,0.5,1.0
1,1.0,0.0,0.0,0.0,1.0,0.5,0.0
2,0.5,1.0,0.5,0.5,0.0,0.0,1.0
3,1.0,0.0,1.0,0.0,0.0,0.5,0.0
4,1.0,0.0,0.5,0.5,1.0,1.0,0.5
...,...,...,...,...,...,...,...
2182,0.0,1.0,1.0,0.5,0.5,1.0,0.0
2183,0.5,0.5,0.0,0.0,1.0,0.5,0.0
2184,0.0,0.0,0.0,1.0,1.0,1.0,0.5
2185,1.0,1.0,1.0,1.0,1.0,0.0,1.0


In [22]:
RuleBase_A=model.predict(rul)

In [23]:
RuleBase_A

array([[0.36604652],
       [0.32206345],
       [0.48865584],
       ...,
       [0.31846502],
       [0.9809755 ],
       [0.3422477 ]], dtype=float32)

In [24]:
Amp=[None]*len(RuleBase_A)
for i in range(0,len(RuleBase_A)):
    if(RuleBase_A[i]<0.333333):
        Amp[i]="L"
    elif(RuleBase_A[i]>0.666666):
        Amp[i]="H"
    else:
        Amp[i]="M"
        
print(Amp.count('L')) 
print(Amp.count('M'))
print(Amp.count('H'))

695
967
525


In [25]:
rul['A']=Amp

In [26]:
rul

,0,1,2,3,4,5,6,A
0,0.0,0.5,0.5,0.5,0.0,0.5,1.0,M
1,1.0,0.0,0.0,0.0,1.0,0.5,0.0,L
2,0.5,1.0,0.5,0.5,0.0,0.0,1.0,M
3,1.0,0.0,1.0,0.0,0.0,0.5,0.0,L
4,1.0,0.0,0.5,0.5,1.0,1.0,0.5,L
...,...,...,...,...,...,...,...,...
2182,0.0,1.0,1.0,0.5,0.5,1.0,0.0,H
2183,0.5,0.5,0.0,0.0,1.0,0.5,0.0,M
2184,0.0,0.0,0.0,1.0,1.0,1.0,0.5,L
2185,1.0,1.0,1.0,1.0,1.0,0.0,1.0,H


In [27]:
rul=rul.replace(to_replace =[0.0, 0.5,1.0],  
                            value =["L","M","H"]) 

In [28]:
rul

,0,1,2,3,4,5,6,A
0,L,M,M,M,L,M,H,M
1,H,L,L,L,H,M,L,L
2,M,H,M,M,L,L,H,M
3,H,L,H,L,L,M,L,L
4,H,L,M,M,H,H,M,L
...,...,...,...,...,...,...,...,...
2182,L,H,H,M,M,H,L,H
2183,M,M,L,L,H,M,L,M
2184,L,L,L,H,H,H,M,L
2185,H,H,H,H,H,L,H,H


In [29]:
rul.to_csv('CNN_Rulebase.csv')